<a href="https://colab.research.google.com/github/heinrichhegenbarth/KAN-CDSCO2004U_Machine_Learning_and_Deep_Learning/blob/main/Ass1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1

## Links
- https://cbscanvas.instructure.com/courses/41482/files/1504373?wrap=1
- https://insideairbnb.com/get-the-data/

## Approach
1. drop qualitative columns
### EDA
2. correlation analysis
3. graphing
### Clustering
4. choose clustering algorithm
5. run clustering algorithm
6. graph clusters
### PCA
7. run pca

In [17]:
# imports
import pandas as pd
import seaborn as sns
import sklearn 



In [18]:
# 1. drop qualitative columns
with open('./data/listings.csv', 'r') as file:
    df_listings = pd.read_csv(file)
    
with open('./data/reviews.csv', 'r') as file:
    df_reviews = pd.read_csv(file)


In [19]:
df_listings.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,3109,https://www.airbnb.com/rooms/3109,20240906025355,2024-09-11,city scrape,zen and calm,Lovely Appartment with one bedroom with a Quee...,Good restaurants<br />very close the Montparna...,https://a0.muscache.com/pictures/miso/Hosting-...,3631,...,5.00,5.00,5.00,7511409139079,t,1,1,0,0,0.05
1,5396,https://www.airbnb.com/rooms/5396,20240906025355,2024-09-13,city scrape,Your perfect Paris studio on Île Saint-Louis,"NEW SOFA-BED SINCE JUNE 2023, Please disregard...","You are within walking distance to the Louvre,...",https://a0.muscache.com/pictures/52413/f9bf76f...,7903,...,4.84,4.96,4.59,7510402838018,f,1,1,0,0,2.23
2,7397,https://www.airbnb.com/rooms/7397,20240906025355,2024-09-06,city scrape,MARAIS - 2ROOMS APT - 2/4 PEOPLE,"VERY CONVENIENT, WITH THE BEST LOCATION !",NaN,https://a0.muscache.com/pictures/67928287/330b...,2626,...,4.89,4.93,4.74,7510400829623,f,1,1,0,0,2.20
3,7964,https://www.airbnb.com/rooms/7964,20240906025355,2024-09-10,previous scrape,Sunny apartment with balcony,"We are renting our a spacious, sunny fully fur...",NaN,https://a0.muscache.com/pictures/miso/Hosting-...,22155,...,5.00,5.00,5.00,7510903576564,f,1,1,0,0,0.03
4,241715,https://www.airbnb.com/rooms/241715,20240906025355,2024-09-11,city scrape,Big Cosy Appartement with 100 m2 Terrace in Paris,Come to stay in our unique Parisian flat to en...,"The 19th arrondissement of Paris, located in t...",https://a0.muscache.com/pictures/miso/Hosting-...,3342097,...,NaN,NaN,NaN,7511913070313,f,1,1,0,0,NaN


In [20]:
df_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,3109,207127433,2017-10-28,51636494,Patricia,Tout s'est bien déroulé. Merci bien. PG
1,3109,208779822,2017-11-03,4142888,Patricia,Un petit nid fouiller douillet situé dans app...
2,3109,295840159,2018-07-24,7415343,Laurent,"Appartement spacieux, propre,clair, et calme à..."
3,3109,553502638,2019-10-24,21159216,Anastasia,"Appartement totalement rénové, en parfait état..."
4,5396,4824,2009-06-30,19995,Sarah,Perfect location!! Nasrine was a delight and m...


In [ ]:
# data cleaning
## drop na prices
df_listings_clean = df_listings.dropna(subset=['price'])
## fill 0 with mean
df_listings_clean['host_response_rate'] = df_listings_clean['host_response_rate'].fillna(df_listings_clean['host_response_rate'].mean())
df_listings_clean['host_acceptance_rate'] = df_listings_clean['host_acceptance_rate'].fillna(df_listings_clean['host_acceptance_rate'].mean())
df_listings_clean['host_listings_count'] = df_listings_clean['host_listings_count'].fillna(df_listings_clean['host_listings_count'].mean())
df_listings_clean['host_total_listings_count'] = df_listings_clean['host_total_listings_count'].fillna(df_listings_clean['host_total_listings_count'].mean())
df_listings_clean['bathrooms'] = df_listings_clean['bathrooms'].fillna(df_listings_clean['bathrooms'].mean())
df_listings_clean['bedrooms'] = df_listings_clean['bedrooms'].fillna(df_listings_clean['bedrooms'].mean())
df_listings_clean['beds'] = df_listings_clean['beds'].fillna(df_listings_clean['beds'].mean())
df_listings_clean['review_scores_rating'] = df_listings_clean['review_scores_rating'].fillna(df_listings_clean['review_scores_rating'].mean())
df_listings_clean['review_scores_accuracy'] = df_listings_clean['review_scores_accuracy'].fillna(df_listings_clean['review_scores_accuracy'].mean())
df_listings_clean['review_scores_cleanliness'] = df_listings_clean['review_scores_cleanliness'].fillna(df_listings_clean['review_scores_cleanliness'].mean())
df_listings_clean['review_scores_checkin'] = df_listings_clean['review_scores_checkin'].fillna(df_listings_clean['review_scores_checkin'].mean())
df_listings_clean['review_scores_communication'] = df_listings_clean['review_scores_communication'].fillna(df_listings_clean['review_scores_communication'].mean())
df_listings_clean['review_scores_location'] = df_listings_clean['review_scores_location'].fillna(df_listings_clean['review_scores_location'].mean())
df_listings_clean['review_scores_value'] = df_listings_clean['review_scores_value'].fillna(df_listings_clean['review_scores_value'].mean())
df_listings_clean['reviews_per_month'] = df_listings_clean['reviews_per_month'].fillna(df_listings_clean['reviews_per_month'].mean())


## cleaning single columns
def clean_price(price):
    return float(price.replace('$', '').replace(',', ''))
df_listings_clean['price'] = df_listings_clean['price'].apply(clean_price)

def clean_perc(perc):
    return float(perc.replace('%', ''))
df_listings_clean['host_response_rate'] = df_listings_clean['host_response_rate'].apply(clean_perc)
df_listings_clean['host_acceptance_rate'] = df_listings_clean['host_acceptance_rate'].apply(clean_perc)

def is_superhost(superhost):
    return 1 if superhost == 't' else 0
df_listings_clean['host_is_superhost'] = df_listings_clean['host_is_superhost'].apply(is_superhost)

## non numeric columns
def drop_non_numeric(df):
    return df.select_dtypes(include=['number'])

df_listings_clean = drop_non_numeric(df_listings_clean)
df_reviews_clean = drop_non_numeric(df_reviews)

## irrelevant columns
drop_cols_listing = ['id', 'scrape_id', 'host_id', 'neighbourhood_group_cleansed', 'calendar_updated']
df_listings_clean = df_listings_clean.drop(columns=drop_cols_listing)

df_listings_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64230 entries, 0 to 95460
Data columns (total 39 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   host_response_rate                            45729 non-null  float64
 1   host_acceptance_rate                          52290 non-null  float64
 2   host_is_superhost                             64230 non-null  int64  
 3   host_listings_count                           64230 non-null  float64
 4   host_total_listings_count                     64230 non-null  float64
 5   latitude                                      64230 non-null  float64
 6   longitude                                     64230 non-null  float64
 7   accommodates                                  64230 non-null  int64  
 8   bathrooms                                     64210 non-null  float64
 9   bedrooms                                      64107 non-null  floa

/var/folders/xt/f477cyyd4y7grcpy4ncx6k6r0000gn/T/ipykernel_65960/475688274.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_listings_clean['price'] = df_listings_clean['price'].apply(clean_price)
/var/folders/xt/f477cyyd4y7grcpy4ncx6k6r0000gn/T/ipykernel_65960/475688274.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_listings_clean['host_response_rate'] = df_listings_clean['host_response_rate'].apply(lambda x: clean_perc(x) if pd.notna(x) else x)
/var/folders/xt/f477cyyd4y7grcpy4ncx6k6r0000gn/

In [ ]:
# Convert column to list
for col in numeric_df.columns:
    
    col_list = numeric_df.loc[:, col].tolist()
## Calculate mean
    sum_col = sum(col_list)
    count_col = len(col_list)
    mean_col = round(sum_col / count_col,6)
        
## Calculate variance
    squared_dist = sum([(mean_col-x)**2 for x in col_list])
    var_col = squared_dist / count_col

## Calculate standard deviation  
    sd_col = var_col ** (1/2)

## print results
    print(f"Mean of {col} is {mean_col}")
    print(f"Variance of {col} is {var_col}")
    print(f"Standard Deviation of {col} is {sd_col}")